In [ ]:
import h2o
from h2o.automl import H2OAutoML

# Initialize H2O cluster
h2o.init()

#importing the required modules
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sympy import symbols, exp, Mul , pi, sqrt, DiracDelta
import seaborn as sns

from sympy.utilities.lambdify import lambdify
import pandas as pd
import csv
from scipy.stats import norm
import math

from google.colab import drive


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.23" 2024-04-16; OpenJDK Runtime Environment (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1); OpenJDK 64-Bit Server VM (build 11.0.23+9-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp03ny97z1
  JVM stdout: /tmp/tmp03ny97z1/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp03ny97z1/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,06 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_unknownUser_pydqgi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
#shiham

drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive/Kaggle

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Kaggle


In [ ]:
pip install h2o

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.3-py2.py3-none-any.whl size=265365897 sha256=dc8f8f85a2433f45cd4a2f2118361e4938b1a89bc32ff39a9086bddf2a0b4b55
  Stored in directory: /root/.cache/pip/wheels/c4/63/41/baa115b5255e1db3e2383bce4e2e6181746aac0b42264c242f
Successfully built h2o


In [ ]:
csv_file = 'final_unnormalized.csv'#make sure to have the Labels column
# csv_file = 'csv_all.csv'
drive.mount('/content/drive')
data = h2o.import_file('/content/drive/MyDrive/Kaggle/final_unnormalized.csv')
#df = pd.read_csv(csv_file)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Load dataset
from h2o.automl import H2OAutoML


# Drop columns
data = data.drop('wind_degree')
data = data.drop('air_quality_us-epa-index')

# Create new columns
data['temp_diff'] = data['feels_like_celsius'] - data['temperature_celsius']
data['wind_diff'] = data['gust_kph'] - data['wind_kph']
# Specify target and features
target = 'condition_text'
features = data.columns
features.remove(target)
data[target] = data[target].asfactor()


H2OValueError: Column(s) selected to drop are not in original frame: 'wind_degree'

In [ ]:

# Split the data
train, test = data.split_frame(ratios=[.8], seed=1234)

# Train AutoML model
aml = H2OAutoML(max_runtime_secs=600, seed=1)
aml.train(x=features, y=target, training_frame=train)

# Evaluate model
lb = aml.leaderboard
print(lb)

best_model = aml.leader
predictions = best_model.predict(test)
performance = best_model.model_performance(test)
print(performance)

# Save the model
#model_path = h2o.save_model(model=best_model, path="/path/to/save", force=True)

# Load the model
#loaded_model = h2o.load_model(model_path)

AutoML progress: |
16:49:58.50: _train param, Dropping bad and constant columns: [city_id_C056, city_id_C078, city_id_C092, day_id, city_id_C038, city_id_C006, city_id_C039]

█
16:50:01.153: _train param, Dropping bad and constant columns: [city_id_C056, city_id_C078, city_id_C092, day_id, city_id_C038, city_id_C006, city_id_C039]

██
16:50:06.456: GLM_1_AutoML_5_20240623_164958 [GLM def_1] failed: java.lang.ArrayIndexOutOfBoundsException
16:50:06.462: _train param, Dropping bad and constant columns: [city_id_C056, city_id_C078, city_id_C092, day_id, city_id_C038, city_id_C006, city_id_C039]

█
16:50:14.346: _train param, Dropping unused columns: [city_id_C056, city_id_C078, city_id_C092, day_id, city_id_C038, city_id_C006, city_id_C039]

██
16:50:20.219: _train param, Dropping bad and constant columns: [city_id_C056, city_id_C078, city_id_C092, day_id, city_id_C038, city_id_C006, city_id_C039]
16:50:25.830: _train param, Dropping bad and constant columns: [city_id_C056, city_id_C078, 

In [ ]:
#drive.mount('/content/drive')
data = h2o.import_file('/content/drive/MyDrive/Kaggle/daily_data.csv_processed.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Drop columns
data = data.drop('wind_degree')
data = data.drop('air_quality_us-epa-index')

# Create new columns
data['temp_diff'] = data['feels_like_celsius'] - data['temperature_celsius']
data['wind_diff'] = data['gust_kph'] - data['wind_kph']

In [ ]:
print(data.shape)

(2893, 128)


In [ ]:
predictions = best_model.predict(data)
print(predictions)

xgboost prediction progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_97e818676d864d126b27d31d13934b9c failed with an exception: java.lang.IllegalArgumentException: Test/Validation dataset has a categorical response column 'condition_text' with no levels in common with the model
stacktrace: 
java.lang.IllegalArgumentException: Test/Validation dataset has a categorical response column 'condition_text' with no levels in common with the model
	at hex.Model.adaptTestForTrain(Model.java:1825)
	at hex.Model.adaptTestForTrain(Model.java:1643)
	at hex.Model.adaptTestForTrain(Model.java:1639)
	at hex.Model.adaptFrameForScore(Model.java:1981)
	at hex.Model.score(Model.java:1999)
	at water.api.ModelMetricsHandler$1.compute2(ModelMetricsHandler.java:555)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1704)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runW